library

In [ ]:
debugMode = False # False # True
num_epochs = 20

import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support


load dataset, preprocess the 4-way classification to 2-way classification.

In [ ]:

raw_data = pd.read_csv(r"/localhome/zpa18/keyLab/CompareNet_FakeNewsDetection/data/fakeNews/fulltrain.csv")    ## Unstemmed
raw_data.columns =['label', 'text']
df = raw_data
idTemp1 = df[(df['label']==1)|(df['label']==4)].index  # |(df.iloc[:, [0]]=='4')
raw_data = df.iloc[idTemp1,:]

split train and test, and transform data structure

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(raw_data.iloc[:, [1]], raw_data.iloc[:, [0]], test_size=0.2, random_state=42)


def transformerFormat(xtrain):
    xtrain = xtrain.to_numpy()
    xtrain = np.reshape(xtrain, (xtrain.shape[0],))
    return xtrain

xtrain = transformerFormat(xtrain)
xtest = transformerFormat(xtest)
ytrain = transformerFormat(ytrain)
ytest = transformerFormat(ytest)

X_train, X_test, y_train, y_test = xtrain, xtest, ytrain, ytest

assign proper labels

In [ ]:

def mapLabel(y_train):
    for i in range(len(y_train)):
        if y_train[i] == 1:  #1 satire
            y_train[i] = 0  #---> 0 satire
        if y_train[i] == 4:  #4 trust
            y_train[i] = 1  # ---> 1  trust
    return y_train


y_train = mapLabel(y_train)
y_test = mapLabel(y_test)


stat of dataset

In [ ]:

NumTrain = len(X_train)
NumTest = len(X_test)
print('NumTrain: %d, NumTest: %d' % (NumTrain, NumTest))

In [ ]:
NumTrain: 19232, NumTest: 4809

Data Loading for Torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

y_train = torch.tensor(y_train).long()
y_test = torch.tensor(y_test).long()

LSTM Model from TextAttack Documents. Including the layers structure, how to encode and decode.

In [ ]:
"""
LSTM 4 Classification
---------------------------------------------------------------------

"""
import json
import os

import torch
from torch import nn as nn

import textattack
from textattack.model_args import TEXTATTACK_MODELS
from textattack.models.helpers import GloveEmbeddingLayer
from textattack.models.helpers.utils import load_cached_state_dict
from textattack.shared import utils

# ==============Using GPU================
if torch.cuda.is_available():
    print('CUDA available, CUDA count: %d, Current Device: %d' % (torch.cuda.device_count(), torch.cuda.current_device()))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class LSTMForClassification(nn.Module):
    """A long short-term memory neural network for text classification.

    We use different versions of this network to pretrain models for
    text classification.
    """

    def __init__(
        self,
        hidden_size=150,
        depth=1,
        dropout=0.3,
        num_labels=2,               ### 2 or 4 classes
        max_seq_length=128,
        model_path=None,
        emb_layer_trainable=False,  ### True -> False
    ):
        super().__init__()
        self._config = {
            "architectures": "LSTMForClassification",
            "hidden_size": hidden_size,
            "depth": depth,
            "dropout": dropout,
            "num_labels": num_labels,
            "max_seq_length": max_seq_length,
            "model_path": model_path,
            "emb_layer_trainable": emb_layer_trainable,
        }
        if depth <= 1:
            # Fix error where we ask for non-zero dropout with only 1 layer.
            # nn.module.RNN won't add dropout for the last recurrent layer,
            # so if that's all we have, this will display a warning.
            dropout = 0
        self.drop = nn.Dropout(dropout)
        self.emb_layer_trainable = emb_layer_trainable
        self.emb_layer = GloveEmbeddingLayer(emb_layer_trainable=emb_layer_trainable)
        self.word2id = self.emb_layer.word2id
        self.encoder = nn.LSTM(
            input_size=self.emb_layer.n_d,
            hidden_size=hidden_size // 2,
            num_layers=depth,
            dropout=dropout,
            bidirectional=True,
        )
        d_out = hidden_size
        self.out = nn.Linear(d_out, num_labels)
        self.tokenizer = textattack.models.tokenizers.GloveTokenizer(
            word_id_map=self.word2id,
            unk_token_id=self.emb_layer.oovid,
            pad_token_id=self.emb_layer.padid,
            max_length=max_seq_length,
        )

        if model_path is not None:
            self.load_from_disk(model_path)
        # self.train()

    def load_from_disk(self, model_path):
        self.load_state_dict(load_cached_state_dict(model_path))
        self.eval()

    def save_pretrained(self, output_path):
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        state_dict = {k: v.cpu() for k, v in self.state_dict().items()}
        torch.save(
            state_dict,
            os.path.join(output_path, "pytorch_model.bin"),
        )
        with open(os.path.join(output_path, "config.json"), "w") as f:
            json.dump(self._config, f)

    @classmethod
    def from_pretrained(cls, name_or_path):
        """Load trained LSTM model by name or from path.

        Args:
            name_or_path (:obj:`str`): Name of the model (e.g. "lstm-imdb") or model saved via :meth:`save_pretrained`.
        Returns:
            :class:`~textattack.models.helpers.LSTMForClassification` model
        """
        # if name_or_path in TEXTATTACK_MODELS:
        #     # path = utils.download_if_needed(TEXTATTACK_MODELS[name_or_path])
        #     path = utils.download_from_s3(TEXTATTACK_MODELS[name_or_path])
        # else:
        #     path = name_or_path
        path = name_or_path
        config_path = os.path.join(path, "config.json")

        if os.path.exists(config_path):
            with open(config_path, "r") as f:
                config = json.load(f)
        else:
            # Default config
            config = {
                "architectures": "LSTMForClassification",
                "hidden_size": 150,
                "depth": 1,
                "dropout": 0.3,
                "num_labels": 2,
                "max_seq_length": 128,
                "model_path": None,
                "emb_layer_trainable": True,
            }
        del config["architectures"]
        model = cls(**config)
        state_dict = load_cached_state_dict(path)
        model.load_state_dict(state_dict)
        return model

    def forward(self, _input):  ##input may be a single-input tuple (text,) or multi-input tuple (text, text, ...). Can process raw text sequences 
        # ensure RNN module weights are part of single contiguous chunk of memory
        self.encoder.flatten_parameters()

        emb = self.emb_layer(_input.t())
        emb = self.drop(emb)

        output, hidden = self.encoder(emb)
        output = torch.max(output, dim=0)[0]

        output = self.drop(output)
        pred = self.out(output)
        return pred

    def get_input_embeddings(self):
        return self.emb_layer.embedding



Training LSTM Model on Our Dataset

load the model structure

In [ ]:

model = LSTMForClassification().to(device)
tokenizer = model.tokenizer
model.train()
print(model)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0001)


model hierarchy:
    
LSTMForClassification(
  (drop): Dropout(p=0, inplace=False)
  (emb_layer): GloveEmbeddingLayer(
    (embedding): Embedding(400001, 200)
  )
  (encoder): LSTM(200, 75, bidirectional=True)
  (out): Linear(in_features=150, out_features=2, bias=True)
)

prepare train and test data

In [ ]:

if debugMode:
    num_epochs = 2
# X_train = X_train[:100]
# y_train = y_train[:100]
# X_test = X_test[:50]
# y_test = y_test[:50] 
# NumTrain = 100
# NumTest = 50

X_train, X_test, y_train, y_test = X_train, X_test, y_train.to(device), y_test.to(device)

X_train, X_test = tokenizer.batch_encode(X_train), tokenizer.batch_encode(X_test)
X_train = torch.tensor(X_train).long().to(device)
X_test = torch.tensor(X_test).long().to(device)

data loader module

In [ ]:

from torch.utils.data import DataLoader, Dataset
class Build_Data(Dataset):    
    # Constructor
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.len = self.x.shape[0]        
    # Getting the data
    def __getitem__(self, index):    
        return self.x[index], self.y[index]    
    # Getting length of the data
    def __len__(self):
        return self.len

# Creating DataLoader object
training_set = Build_Data(X_train, y_train)
train_loader = DataLoader(dataset = training_set, batch_size = 128)
test_set = Build_Data(X_test, y_test)
test_loader = DataLoader(dataset = test_set, batch_size = 128)



training process and testing process

In [ ]:

for epoch in range(num_epochs):
    loss_epoch = 0
    acc_epoch = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        loss_epoch += loss.item()
        optimizer.step()

        _, prediction = torch.max(F.softmax(outputs, dim=1), 1)
        pred_y = prediction.data.cpu().numpy().squeeze()
        target_y = y.data.cpu().numpy()
        acc_epoch += sum(pred_y == target_y)


    print(f"Epoch {epoch+1} loss: {loss_epoch/NumTrain} \t Train Acc: {acc_epoch/NumTrain}")

    accuracy = acc_epoch/NumTrain
    prec_mac, recall_mac, f1_mac, _ = precision_recall_fscore_support(target_y, pred_y, average='macro')
    prec_mic, recall_mic, f1_mic, _ = precision_recall_fscore_support(target_y, pred_y, average='micro')
    print("Accuracy on the OOD test set 1: {:.4f}".format(accuracy))
    print("Precision on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(prec_mac, prec_mic))
    print("Recall on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(recall_mac, recall_mic))
    print("F1 on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(f1_mac, f1_mic))
    print("Latex: {:5.2f} & {:5.2f} & {:5.2f} & {:5.2f}".format(accuracy * 100, prec_mac * 100, recall_mac * 100,
                                                                f1_mac * 100))
    print("----------------------------------------------------------------------")


    with torch.no_grad():
        acc_test_epoch = 0
        for x, y in test_loader:
            test_out = model(x)
            _, prediction = torch.max(F.softmax(test_out, dim=1), 1)
            pred_y = prediction.data.cpu().numpy().squeeze()
            target_y = y.data.cpu().numpy()
            acc_test_epoch +=  sum(pred_y == target_y) 
        print('--  test accuracy', acc_test_epoch/NumTest)
        accuracy = acc_test_epoch/NumTest
        prec_mac, recall_mac, f1_mac, _ = precision_recall_fscore_support(target_y, pred_y, average='macro')
        prec_mic, recall_mic, f1_mic, _ = precision_recall_fscore_support(target_y, pred_y, average='micro')
        print("Accuracy on the OOD test set 1: {:.4f}".format(accuracy))
        print("Precision on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(prec_mac, prec_mic))
        print("Recall on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(recall_mac, recall_mic))
        print("F1 on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(f1_mac, f1_mic))
        print("Latex: {:5.2f} & {:5.2f} & {:5.2f} & {:5.2f}".format(accuracy * 100, prec_mac * 100, recall_mac * 100,
                                                                    f1_mac * 100))
        print("----------------------------------------------------------------------")


In [ ]:
Epoch 1 loss: 0.003916607568421697 	 Train Acc: 0.7466722129783694
Accuracy on the OOD test set 1: 0.7467
Precision on the OOD test set 1 macro / micro: 0.8140, 0.7812
Recall on the OOD test set 1 macro / micro: 0.7579, 0.7812
F1 on the OOD test set 1 macro / micro: 0.7625, 0.7812
Latex: 74.67 & 81.40 & 75.79 & 76.25
----------------------------------------------------------------------
--  test accuracy 0.8442503639010189
Accuracy on the OOD test set 1: 0.8443
Precision on the OOD test set 1 macro / micro: 0.8274, 0.7945
Recall on the OOD test set 1 macro / micro: 0.7665, 0.7945
F1 on the OOD test set 1 macro / micro: 0.7740, 0.7945
Latex: 84.43 & 82.74 & 76.65 & 77.40